Download a list of commissioners from the Office of Advisory Neighborhood Commissions

In [1]:
import os
os.chdir('../')

In [2]:
import time
import requests
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz

from scripts.common import hash_dataframe, current_timestamp, validate_smd_ids
from scripts.data_transformations import list_commissioners

pd.set_option('display.max_rows', 500)

In [3]:
ancs = pd.read_csv('data/ancs.csv')
ancs = ancs[ancs.redistricting_year == 2022].copy()

In [ ]:
comm_dict = {}

for idx, anc in tqdm(ancs.iterrows(), total=len(ancs)):
    r = requests.get(anc.dc_oanc_link)
    
    table_list = pd.read_html(r.text, converters={'SMD': str})

    if 'SMD' in table_list[0].columns:
        comm_dict[anc.anc_id] = table_list[0].copy()
    elif anc.anc_id == 'anc_5F_2022':
        # First row of this dataframe is the column names and should be fixed then dropped
        comm_dict[anc.anc_id] = table_list[0].copy()
        comm_dict[anc.anc_id].columns = comm_dict[anc.anc_id].iloc[0]
        comm_dict[anc.anc_id].drop(0, inplace=True)
    else:
        print(f'Warning: ANC {anc.anc_id} table not present on OANC site.')
        print(table_list)
    
    time.sleep(2)

 28%|███████████████████████▏                                                          | 13/46 [00:31<01:15,  2.28s/it]

In [ ]:
comm_official = pd.concat(comm_dict).reset_index().rename(columns={'level_0': 'anc_id'}).drop(columns='level_1')

In [ ]:
comm_official['smd_id'] = 'smd_2022_' + comm_official['SMD']

# Fix some mangled SMD IDs
comm_official['smd_id'] = (
    comm_official['smd_id']
    .str.replace('smd_2022_3G', 'smd_2022_3/4G')
    .str.replace('smd_2022_7B02l', 'smd_2022_7B02')
)


In [ ]:
# Confirm that district list is correct
validate_smd_ids(comm_official)
assert len(comm_official) == 345

In [ ]:
comm_official['is_vacant'] = comm_official.Name.str.lower()== 'vacant'
comm_official['is_chairperson'] = comm_official.Name.str.lower().str.contains('chairperson|chairpeson')

In [ ]:
comm_official['oanc_name'] = (
    comm_official.Name
    .str.replace(' Chairperson', '')
    .str.replace(' Chairpeson', '')
    .str.replace(' Vice Chair/Treasurer', '')
    .str.replace(' Vice Chair', '')
    .str.replace(' Secretary', '')
    .str.replace(' Parlimentarian', '')
    .str.replace(' Sargent-at-arms', '')
    .str.strip()
)

In [ ]:
comm_official['oanc_hash_id'] = hash_dataframe(comm_official, ['SMD', 'oanc_name'])

In [ ]:
# Compare official to OpenANC
comm_openanc = list_commissioners(status='current')
comm = pd.merge(comm_official, comm_openanc, how='left', on='smd_id')

comm['oanc_name'] = comm['oanc_name'].fillna('Vacant')
comm['openanc_name'] = comm['commissioner_name'].fillna('Vacant')
comm['name_score'] = comm.apply(lambda x: fuzz.ratio(x.oanc_name, x.openanc_name), axis=1)

In [ ]:
"""
0 = the district is not vacant according to both sources (good)
1 = the district is vacant according to one source (bad)
2 = the district is vacant according to both sources (good)
"""
comm['num_sources_say_vacant'] = (comm[['oanc_name', 'openanc_name']] == 'Vacant').sum(axis=1)
comm.loc[comm['num_sources_say_vacant'] == 1, 'name_score'] = 0

In [ ]:
# Evaluate name mismatches
comm.loc[comm.name_score < 100, ['smd_id', 'oanc_name', 'openanc_name', 'name_score']].sort_values(by='name_score')

In [ ]:
columns_to_save = [
    'smd_id'
    , 'Name'
    , 'oanc_name'
    , 'is_vacant'
    , 'is_chairperson'
    , 'oanc_hash_id'
]

filename = 'data/oanc/commissioners_{}.csv'.format(current_timestamp().strftime('%Y-%m-%d'))

comm_official[columns_to_save].to_csv(filename, index=False)

In [ ]:
# Confirm that there is one chairperson per ANC
comm_official['is_chairperson_int'] = comm_official['is_chairperson'].astype(int)
num_chairs = comm_official.groupby('anc_id').is_chairperson_int.sum()
# num_chairs.sum() == comm_official.anc_id.nunique()

# Chairs have not yet been noted for the 2023 session
num_chairs.sum()

In [ ]:
# Districts with an issue to resolve
comm.loc[comm['num_sources_say_vacant'] == 1, ['smd_id', 'openanc_name']] #.to_clipboard(index=False)

In [ ]:
r.text

In [ ]:
comm_dict['anc_8E_2022']